In [13]:
import os
import csv
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from champion_net import ChampionNet
import torch.optim as optim
import time
import random
import matplotlib.pylab as plt

#config
config = {
    'data_set': '../../data/filtered-dataset-no-header.csv',
    'model': {
        'save': False,
        'save_location': '../models',
        'layer_size': [32, 64, 128, 256],
        'dropout_rate': [0.5],
        'learning_rate': [0.0005, 0.005, 0.01]
    },
    'batch_size': 50000,
    'epochs': [10,15,20],
    'validation_set_size': 0.1,
    'test_set_size': 0.1,
    'lambda1': 0.5,
    'lambda2' : 0.01,
    'show_graphs': False,
    'graph_locations': '../graphs',
    'device': 'cpu'
}

device = torch.device(config['device'])

In [14]:
#training the nn functions
def accuracy(outputs, y):
    matches = [torch.argmax(i, 0) == torch.argmax(j,0) for i,j in zip(outputs,y)]
    return matches.count(True)/len(matches)
    
def forward_pass(net, X,y,train = False):
    if train:
        optimizer.zero_grad()
    outputs = net(X)
    acc = accuracy(outputs, y)
    cross_entropy_loss = loss_function(outputs,y)

    all_linear2_params = torch.cat([x.view(-1) for x in net.fc2.parameters()])
    l2_regularization = lambda2 * torch.norm(all_linear2_params, 2)

    loss = cross_entropy_loss + l2_regularization

    if train:
        loss.backward()
        optimizer.step()
    return acc, loss

def test(net, val_X, val_y, size=32):
    X, y = val_X[:size], val_y[:size]
    val_acc, val_loss = forward_pass(net, X.view(-1,154).to(device), y.to(device).view(-1,2))
    return val_acc, val_loss

def train(net, train_X, train_y,model_name, epochs):
    log = {
        'times': [],
        'acc': [],
        'loss': [],
        'val_acc': [],
        'val_loss': [],
    }
    for epoch in range(epochs):
        for i in tqdm(range(0, len(train_X), config['batch_size'])): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:100] ..for now just to dev
            batch_X = train_X[i:i+config['batch_size']].view(-1,154)
            batch_y = train_y[i:i+config['batch_size']].view(-1, 2)
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            acc, loss = forward_pass(net, batch_X, batch_y, train=True)
        
            if i % config['batch_size'] == 0:
                val_acc, val_loss = test(net, val_X, val_y, size=config['batch_size'])
                log['times'].append(time.time())
                log['acc'].append(round(float(acc),2))
                log['loss'].append(round(float(loss), 4))
                log['val_acc'].append(round(float(val_acc),2))
                log['val_loss'].append(round(float(val_loss),4))
    return log

def create_acc_loss_graph(training_results, model_name, save_graphs, show_graphs):
    times = [time - training_results['times'][0] for time in training_results['times']]
    data = [training_results['acc'],training_results['loss'],training_results['val_acc'],training_results['val_loss']]
    y_labels = ['accuracy', 'loss', 'validation_accuracy', 'validation_loss']

    for i in range(len(data)):
        create_and_save_graph(times, data[i], 'iterations', y_labels[i], model_name, save_graphs, show_graphs)

def create_and_save_graph(x, y, x_label, y_label, model_name, save_graph, show_graph):
    plt.plot(x, y)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(y_label)
    if save_graph:
         plt.savefig(f"{config['model']['save_location']}/{model_name}/{y_label}.png", format='png', bbox_inches='tight')
    if show_graph:
        plt.show()


In [15]:
#useful helper functions
def convert_to_state(combination):
    res = [0 for i in range(154)] #154 champions
    for i in range(5):
        res[int(combination[i])] = 1
    
    for i in range(5, 10):
        res[int(combination[i])] = -1
    return res


def create_features_and_labels(dataset):
    features = []
    labels = []
    for point in dataset:
        feature_state = convert_to_state([int(x) for x in point[0][:10]])
        features.append(feature_state)
        blue_victory = int(point[0][10]) #1 if blue victory, 0 otherwise
        labels.append([blue_victory, 1 - blue_victory])
    
    return torch.Tensor(features), torch.Tensor(labels)

def load_data_set(filename, validation_set_size,test_set_size, delimiter=','):
    dataset = []
    with open(filename, 'r') as f:
        reader = csv.reader(f, delimiter=delimiter)
        for row in reader:
            dataset.append(np.array([row]))


    print(f'The size of the entire dataset is {len(dataset)} points')
    val_size = int(len(dataset) * validation_set_size)
    test_size = int(len(dataset) * test_set_size)
    train_size = len(dataset) - val_size - test_size
    random.shuffle(dataset)

    train_set = dataset[:train_size]
    val_set = dataset[train_size:(val_size + train_size)]
    test_set = dataset[(val_size + train_size):(val_size + train_size+test_size)]

    return train_set, val_set, test_set

def save_trained_network(net, net_name, location):
    os.mkdir(f'{location}/{net_name}')
    path = f'{location}/{net_name}/model.pickle'
    torch.save(net.state_dict(), path)

In [16]:
    
#Generate the validation and training sets
training_set, validation_set, test_set = load_data_set(config['data_set'], config['validation_set_size'], config['test_set_size'])

train_X, train_y = create_features_and_labels(training_set)
val_X, val_y = create_features_and_labels(validation_set)
test_X, test_y = create_features_and_labels(test_set)

The size of the entire dataset is 966973 points


In [17]:
best_params = {
    "layer_size": 0,
    "epochs": 0, 
    "learning_rate": 0 ,
}

best_test_acc = -1

In [18]:
#perform the grid search
for layer_size in  config['model']['layer_size']:
    for epoch in config['epochs']:
        for lr in config['model']['learning_rate']:
            for dropout_rate in config['model']['dropout_rate']:
                lambda1 = config['lambda1']
                lambda2 = config['lambda2'] 

                model_name = f"champion-model-{int(time.time())}-{layer_size}-{epoch}-{lr}"
                
                net = ChampionNet(num_units=layer_size).to(device)
                
                optimizer = optim.Adam(net.parameters(),lr = lr)
                
                loss_function = nn.BCEWithLogitsLoss()
                
                res = train(net, train_X, train_y,model_name, epoch)

                #save the model, if configured to do so
                if(config['model']['save']):
                    save_trained_network(net, model_name, config['model']['save_location'])

                #make the graphs
                create_acc_loss_graph(res, model_name, config['model']['save'], config['show_graphs'])

                #run on the test set to determine accuracy
                test_acc, test_loss = test(net, test_X, test_y, size=len(test_X))
                print(f'Test accuracy of model with hyperparameters: {layer_size}, {epoch}, {lr}: {test_acc}')
                if test_acc > best_test_acc:
                    best_params['layer_size'] = layer_size
                    best_params['epochs'] = epoch
                    best_params['learning_rate'] = lr
                    best_test_acc = test_acc

 81%|████████▏ | 13/16 [00:36<00:08,  2.79s/it]


KeyboardInterrupt: 